In [1]:
%%writefile assign4.cpp
#include<bits/stdc++.h>
#include "omp.h"

using namespace std;
list<int>q;
vector<int>w(7,1000);
int visited[7];
int local_q;

struct Comp
{
    // Compare 2 Player objects using name
    bool operator ()(const int & a, const int & b)
    {
        return w[a]<w[b];
    }
};

void showlist(list <int> g) 
{ 
    list <int> :: iterator it; 
    for(it = g.begin(); it != g.end(); ++it) 
        cout<<*it<<" "; 
    cout<<endl; 
}

void showvec(vector <int> g) 
{ 
    vector <int> :: iterator it; 
    for(it = g.begin(); it != g.end(); ++it) 
        cout<<*it<<" "; 
    cout<<endl; 
}

void bfs(int adj_matrix[7][7],  int n_nodes)
{
		if(q.empty())
			return;
    q.sort(Comp());
    //showlist(q);
    //showvec(w);
		//pop first element
		int cur_node = q.front();
    q.pop_front();
		
		printf("%d, ", cur_node);
		omp_set_num_threads(3);
		#pragma omp parallel for shared(visited)
		for(int i=0; i<n_nodes; i++)
		{
			if(adj_matrix[cur_node][i]>0 && visited[i]==0)
			{
        if(w[i] > adj_matrix[cur_node][i]){
            w[i] = adj_matrix[cur_node][i];
            //cout<<i<<" "<<w[i]<<endl;
        }
				q.push_back(i);
				visited[i]=1;
			}
		}
	bfs(adj_matrix,  n_nodes);
}

int main()
{
  int th = omp_get_max_threads();
  cout<<"Max Threads : "<<th<<endl;

	int n_nodes = 7;
	for(int i=0; i<n_nodes; i++)
	{
		visited[i] = 0;
	}
	//shows graph connections
	int adj_matrix[7][7] = {
							{0,  10  ,20  ,0  ,0  ,0  ,0},
							{10	,0	,30	,30	,0	,0	,0},
							{20	,30	,0	,0	,40	,0	,0},
							{0	,30	,0	,0	,20	,0	,0},
							{0	,0	,40	,20	,0	,10	,0},
							{0	,0	,0	,0	,10	,0	,20},
							{0	,0	,0	,0	,0	,20	,0}
							};

	int start_node = 3;	//set start node as 1
	q.push_back(start_node);
    w[start_node] = 0;
	visited[start_node] = 1;
	
	//int cur_node;
	bfs(adj_matrix,  n_nodes);

	return 0;
}

Writing assign4.cpp


In [2]:
%%script bash

g++ assign4.cpp -fopenmp -o assign
./assign

Max Threads : 2
3, 4, 5, 6, 1, 0, 2, 

In [7]:
%%writefile assign4.cpp
#include<iostream>
#include<omp.h>
#include<time.h>
using namespace std;
int c1[3];
int binary_search(int a[],int low,int high,int key){
  
  int loc = -1;
  int mid;
  while(low<=high){
    mid = (high + low )/2;
    if (a[mid] == key) {
      loc = mid;
      break;
    }
    else {
      #pragma omp parallel sections
      {
        //cout<<omp_get_thread_num();
        #pragma omp section
        {
          if(a[mid]<key){
            low = mid+1;
          }
        }
        #pragma omp section
        {
          if(a[mid]>key){
            high = mid-1;
          }
        }
      }
    }
  }
  return loc;
}

int main(){
  int th = omp_get_max_threads();
  cout<<"Max Threads : "<<th<<endl;
  omp_set_num_threads(2);
  c1[1]=0;
  c1[2]=0;
  int a[1000000];
  clock_t t1,t2;
  int key = 0;
  int loc,i;
  for (int i = 0; i < 1000000; i++) {
    a[i] = i;
  }
  key = 10000;
  t1 = clock();
  loc = binary_search(a,0,1000000,key);
  t2 = clock();
  if (loc == -1) {
    cout<<"Not Found";
  } else {
    cout<<"Found at index = "<<loc<<endl;
    //cout<<"By Thread "<<omp_get_thread_num()<<endl;
  }
  return 0 ;
}

Overwriting assign4.cpp


In [8]:
%%script bash

g++ assign4.cpp -fopenmp -o assign
./assign

Max Threads : 2
Found at index = 10000
